In [95]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler, QuantileTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, balanced_accuracy_score

model_df = pd.read_csv('../Data/model_data.csv', index_col = 0)
model_df = model_df.drop('industry', axis = 1)
# modle_df = model_df.set_index('office', inplace = True)
model_df.head(3)

,current_ratio,operating_cash_flow,debt_to_equity,interest_coverage,operating_margin,return_on_assets,return_on_equity,office,has_interest_payments
0001090872:2011:FY,3.031573,0.477953,1.098471,12.453488,0.533485,0.111737,0.234476,Industrial Applications and Services,1
0001090872:2012:FY,2.445325,-0.621236,1.032015,11.079208,0.525518,0.109434,0.222372,Industrial Applications and Services,1
0001090872:2013:FY,3.110487,0.202247,1.020420,8.887850,0.521233,0.067752,0.136888,Industrial Applications and Services,1


In [96]:
target_encoding = LabelEncoder()
scaling = QuantileTransformer()

X = scaling.fit_transform(X)

X = model_df.drop('office', axis = 1).copy()

y = model_df['office'].copy()

y = target_encoding.fit_transform(y)

office_dict = {}

for label in range(len(target_encoding.classes_)):
    office = target_encoding.classes_[label]
    office_dict[label] = office
    
office_dict

{0: 'Industrial Applications and Services',
 1: 'Office of Crypto Assets',
 2: 'Office of Energy & Transportation',
 3: 'Office of Finance',
 4: 'Office of Finance or Office of Crypto Assets',
 5: 'Office of Life Sciences',
 6: 'Office of Manufacturing',
 7: 'Office of Real Estate & Construction',
 8: 'Office of Technology',
 9: 'Office of Trade & Services'}

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [100]:
model = SVC()

# parameters = {'loss': (False, True), 'C': (1, 5, 10), 'class_weight': (None, 'balanced'), 'max_iter': (1000, 1500, 2500)}
# model.fit(X_train, y_train)
parameters = {'C': [1], 'class_weight': (None, 'balanced')}

predictor = GridSearchCV(model, parameters, scoring = 'balanced_accuracy', n_jobs = 2, verbose = 3)

predictor.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


GridSearchCV(estimator=SVC(), n_jobs=2,
             param_grid={'C': [1], 'class_weight': (None, 'balanced')},
             scoring='balanced_accuracy', verbose=3)

In [101]:
predictor.best_estimator_

SVC(C=1)

In [102]:
model = predictor.best_estimator_
y_pred = model.predict(X_test)
y_pred
accuracy_score(y_test, y_pred)

0.1807399539741547

In [105]:
model_test = SVC(C = 1)

model_test.fit(X_train, y_train)

y_pred_test = model_test.predict(X_test)

print(accuracy_score(y_test, y_pred_test))
print(balanced_accuracy_score(y_test, y_pred, adjusted = False))

0.1807399539741547
0.10065520193328044


In [79]:
y_pred = model.predict(X_test)
y_pred

NotFittedError: This SVC instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
accuracy_score(y_test, y_pred)

0.17365905469994689

In [70]:
balanced_accuracy_score(y_test, y_pred, adjusted = False)

0.1197195305042787